In [2]:
# Modules 

import numpy as np
import pandas as pd

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 1700)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from datetime import date

In [ ]:
d = str(date.today())

deliverable_with_stealths_labeled_adi = 'data/FW20_forecasts_adi_' + d + '.xlsx'
deliverable_with_stealths_labeled_rbk = 'data/FW20_forecasts_rbk_' + d + '.xlsx'

In [7]:
# both = pd.read_excel(deliverable_with_stealths_labeled_adi, index_col=0)
both = pd.read_excel(deliverable_with_stealths_labeled_rbk, index_col=0)

In [10]:
both = both[['net_forecast', 'buy_recommendation', 'net_forecast_w_clearance', 'buy_recommendation_w_clearance']].copy()
both['change'] = both.net_forecast_w_clearance - both.net_forecast

In [ ]:
# --- Compare to observed with-clearance net_qty for EDA ---
articles_of_interest = np.load('articles_of_interest.npy', allow_pickle=True).tolist()

dat0 = pd.read_csv('data/ch4k.csv')
dat0 = dat0[dat0.season.isin(['FW19'])]

# --- clearance/non-clearance --- 
remove_clearance = False

if(remove_clearance):
    dat0['clearance'] = dat0.clearance.fillna(0) 
    dat0['net_qty'] = (1 - dat0.clearance)*dat0.net_qty
    print('clearance_removed')
else:
    print('include_clearance')
    
dat = dat0[dat0.article_number.isin(articles_of_interest)].copy()

In [23]:
both2 = pd.merge(
    both, 
    pd.DataFrame(dat.groupby('article_number')['net_qty'].sum()).reset_index(), # add observed w/ clearance net_qty for EDA
)

both2.sort_values('change', ascending=False)

,article_number,net_forecast,buy_recommendation,net_forecast_w_clearance,buy_recommendation_w_clearance,change,net_qty
1277,F35556,10245,12283,29968,35106,19723,21912
517,D95958,5419,7151,24922,30788,19503,21646
1214,EF0213,2807,3688,18429,21771,15622,12962
1391,S21489,23053,27267,37928,44568,14875,33935
827,DV1560,4943,6505,17596,21725,12653,12139
563,DP2389,7916,9986,20180,24629,12264,14413
1392,S21490,14711,17562,26144,30857,11433,23767
562,DP2386,7338,9238,17068,20801,9730,12807
1009,DZ9357,12711,15241,22433,26553,9722,19114
694,DT7964,6316,8054,15582,19096,9266,11968


# ----- Compare to originals -----

In [29]:
old = pd.read_excel('FW20_forecasts_incl_cl.xlsx')
old.columns

Index(['article_number', 'brand', 'model_no', 'description', 'type', 'BU',
       'RMH', 'price', 'cost', 'margin', 'net_forecast', 'buy_recommendation',
       'net_forecast_incl_clearance', 'buy_rec_incl_clearance', 'eCom_21Nov',
       'FW19_total_market_FC', 'FW19_total_ecom_SO', 'FW19_total_ecom_RDP',
       'notes'],
      dtype='object')

In [30]:
old = old[['article_number', 'net_forecast', 'buy_recommendation',
       'net_forecast_incl_clearance', 'buy_rec_incl_clearance']].rename(columns = {
    'net_forecast': 'net_old', 
    'buy_recommendation': 'buy_rec_old',
    'net_forecast_incl_clearance': 'net_incl_cl_old', 
    'buy_rec_incl_clearance': 'buy_rec_incl_cl_old'
})

In [36]:
old.shape


(1065, 5)

In [ ]:
abc = pd.merge(both, old, on = 'article_number', how = 'left')
abc.shape
abc

In [35]:
abc.net_old.isna().sum()

404